In [1]:
import pandas as pd
from datetime import timedelta, datetime

# Load datasets
ila_pd = pd.read_csv("ila_pd.csv")
lakselus = pd.read_csv("lakselus_per_fisk.csv")


C:\Users\Konrad\AppData\Local\Temp\ipykernel_15712\2731569889.py:5: DtypeWarning: Columns (16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  ila_pd = pd.read_csv("ila_pd.csv")


In [2]:
# some preprocessing of data, deleting null
lakselus = lakselus.dropna(subset=['Voksne hunnlus', 'Lus i bevegelige stadier', 'Fastsittende lus'])
ila_pd = ila_pd.dropna(subset=['Fra dato', 'Til dato'])

ila_pd['Fra dato'] = pd.to_datetime(ila_pd['Fra dato'], errors='coerce')
ila_pd['Til dato'] = pd.to_datetime(ila_pd['Til dato'], errors='coerce')

In [3]:
# classifying into quarters based on the year and week
def classify_quarter(year, week):
    # Calculating the date from year and week
    date_str = f'{year}-W{int(week)}-1'
    date = datetime.strptime(date_str, "%Y-W%W-%w")

    # Determining the quarter
    quarter = (date.month - 1) // 3 + 1
    return f'{year}-Q{quarter}'

# Applying the function to the first dataset
lakselus['Quarter'] = lakselus.apply(lambda row: classify_quarter(row['År'], row['Uke']), axis=1)


In [4]:
ila_pd.head()

,Uke,År,Lokalitetsnummer,Lokalitetsnavn,Sykdom,Status,Fra dato,Til dato,Kommunenummer,Kommune,...,Lat,Lon,ProduksjonsområdeId,Produksjonsområde,UtbruddsId,Subtype,Mistanke-dato,Påvist-dato,Tømt-dato,Avsluttet-dato
17109,44,2020,12067,Aldalen,PD,Påvist,2019-07-31,2021-02-03,4624.0,BJØRNAFJORDEN,...,60.249950,5.571917,3.0,Karmøy til Sotra,NaN,NaN,NaN,NaN,NaN,NaN
17112,44,2020,11800,Austneståa,PD,Mistanke,2020-02-21,2021-01-19,4602.0,KINN,...,61.490818,5.171167,4.0,Nordhordland til Stadt,NaN,NaN,NaN,NaN,NaN,NaN
17115,44,2020,22455,Bjørlykkestranda,PD,Mistanke,2020-01-16,2020-12-18,1511.0,VANYLVEN,...,62.166550,5.595400,5.0,Stadt til Hustadvika,NaN,NaN,NaN,NaN,NaN,NaN
17116,44,2020,22455,Bjørlykkestranda,ILA,Påvist,2020-09-24,2020-12-18,1511.0,VANYLVEN,...,62.166550,5.595400,5.0,Stadt til Hustadvika,NaN,NaN,NaN,NaN,NaN,NaN
17117,44,2020,12308,Bjørndal,PD,Mistanke,2020-03-05,2020-11-21,1520.0,ØRSTA,...,62.274883,6.002850,5.0,Stadt til Hustadvika,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# for the ila dataset we are taking into consideration how long the disease varied into quarters
def get_quarter_boundaries(year, quarter):
    start_month = 3 * quarter - 2
    end_month = 3 * quarter
    start_date = datetime(year, start_month, 1)
    if end_month == 12:
        end_date = datetime(year, end_month, 31)
    else:
        end_date = datetime(year, end_month + 1, 1) - timedelta(days=1)
    return start_date, end_date

# Updated function to handle short spans within a single quarter
def classify_quarter_majority_improved(from_date, to_date):
    quarters = []

    if pd.notna(from_date) and pd.notna(to_date):
        # Check if the entire span is within a single quarter
        start_quarter = (from_date.month - 1) // 3 + 1
        end_quarter = (to_date.month - 1) // 3 + 1
        if start_quarter == end_quarter and from_date.year == to_date.year:
            quarters.append(f'{from_date.year}-Q{start_quarter}')
            return quarters

        # Iterate through each year and quarter in the range
        current_date = from_date
        while current_date <= to_date:
            year = current_date.year
            quarter = (current_date.month - 1) // 3 + 1
            quarter_start, quarter_end = get_quarter_boundaries(year, quarter)

            # Adjust the quarter end date to the end of the span if it's earlier
            quarter_end = min(quarter_end, to_date)

            # Calculate the duration of the span within the quarter
            duration_in_quarter = (quarter_end - current_date).days + 1

            # Calculate the total duration of the quarter
            total_quarter_duration = (quarter_end - quarter_start).days + 1

            # Check if the majority of the span is in this quarter
            if duration_in_quarter >= total_quarter_duration / 2:
                quarters.append(f'{year}-Q{quarter}')

            # Move to the next quarter
            current_date = quarter_end + timedelta(days=1)

    return quarters

# Applying the quarter classification function to the second dataset
ila_pd['Quarter'] = ila_pd.apply(lambda row: classify_quarter_majority_improved(row['Fra dato'], row['Til dato']), axis=1)

# then I am exploding quarter field, as it contains array of values, so now each row has only one value in quarter
ila_pd = ila_pd.explode('Quarter')

In [7]:
# Selecting relevant columns from both datasets
# taking into consideration only diseases that are proven
ila_pd = ila_pd[ila_pd['Status'] == 'Påvist']

# From the ila_df we taking place number, quarter (time) and type of disease
ila_df_selected = ila_pd[['Lokalitetsnummer', 'Quarter', 'Sykdom']]

# From lakselus df we take place number, quarter (time) and three types of fish lice
lakselus_df_selected = lakselus[['Lokalitetsnummer', 'Quarter', 'Voksne hunnlus', 'Lus i bevegelige stadier', 'Fastsittende lus']]

# Merging the datasets based on 'Lokalitetsnummer' and 'Quarter'
merged_df = pd.merge(ila_df_selected, lakselus_df_selected, on=['Lokalitetsnummer', 'Quarter'], how='inner')

# Dropping rows where 'Sykdom', 'Voksne hunnlus', 'Lus i bevegelige stadier', or 'Fastsittende lus' are zero or NaN
merged_df = merged_df[
    (merged_df['Sykdom'] != 0) &
    (merged_df['Voksne hunnlus'] != 0) &
    (merged_df['Lus i bevegelige stadier'] != 0) &
    (merged_df['Fastsittende lus'] != 0)
].dropna(subset=['Sykdom', 'Voksne hunnlus', 'Lus i bevegelige stadier', 'Fastsittende lus'])

# change columns names into english ones
merged_df.rename(columns={
    'Lokalitetsnummer': 'LocalityNumber',
    'Quarter': 'Quarter',
    'Sykdom': 'Disease',
    'Voksne hunnlus': 'Adult Female Lice',
    'Lus i bevegelige stadier': 'Movable Lice',
    'Fastsittende lus': 'Attached Lice'
}, inplace=True)

# Displaying the first few rows of the merged and filtered dataset
merged_df.head()


,LocalityNumber,Quarter,Disease,Adult Female Lice,Movable Lice,Attached Lice
1,12067,2019-Q3,PD,0.01,0.12,0.03
2,12067,2019-Q3,PD,0.03,0.09,0.03
3,12067,2019-Q3,PD,0.03,0.11,0.03
4,12067,2019-Q3,PD,0.07,0.11,0.03
5,12067,2019-Q3,PD,0.05,0.17,0.07


In [19]:
# # Choosing place
# location_counts = lakselus['Lokalitetsnummer'].value_counts()
# top_location_number = location_counts.idxmax()
# number_of_rows = location_counts.max()

# top_location_number, number_of_rows
# Determine the counts of Lokalitetsnummer in both dataframes
lakselus_counts = lakselus['Lokalitetsnummer'].value_counts()
ila_counts = ila_pd['Lokalitetsnummer'].value_counts()

# Create a DataFrame from the intersection of both series
combined_counts_df = pd.concat([lakselus_counts, ila_counts], axis=1, keys=['Lakselus', 'Ila_pd']).fillna(0)
combined_counts_df['Total'] = combined_counts_df['Lakselus'] + combined_counts_df['Ila_pd']

# Find the Lokalitetsnummer with the maximum total count
top_combined_lokalitetsnummer = combined_counts_df['Total'].idxmax()
max_combined_count = combined_counts_df['Total'].max()

top_combined_lokalitetsnummer, max_combined_count

(18275, 976.0)

In [52]:
# choosing one location for analysis

specific_lokalitetsnummer = top_combined_lokalitetsnummer

ila_pd = ila_pd[ila_pd['Lokalitetsnummer'] == specific_lokalitetsnummer]
lakselus = lakselus[lakselus['Lokalitetsnummer'] == specific_lokalitetsnummer]


In [8]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

columns_to_bin = ['Adult Female Lice', 'Movable Lice', 'Attached Lice']
for column in columns_to_bin:
    if column in merged_df.columns:
        # Normalize the column
        scaled_data = scaler.fit_transform(merged_df[[column]])

        # Determine bin edges based on percentiles
        # You can adjust these percentiles as needed for your data distribution
        percentiles = [0, 25, 50, 75, 95, 100]
        bin_edges = np.percentile(scaled_data, percentiles)

        # Assign values to bins
        merged_df[column + ' Category'] = pd.cut(scaled_data.flatten(), bins=bin_edges, include_lowest=True, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

        # Printing the actual bin edges
        print(f"Actual bin edges for {column}: {bin_edges}")
    else:
        print(f"Column {column} not found in DataFrame.")

# Drop the original numerical columns
merged_df.drop(columns=columns_to_bin, inplace=True)

# Display the first few rows of the binned dataset
merged_df.head()


Actual bin edges for Adult Female Lice: [0.         0.00645161 0.0156682  0.03041475 0.07281106 1.        ]
Actual bin edges for Movable Lice: [0.         0.0054188  0.01238582 0.02678433 0.07307633 1.        ]
Actual bin edges for Attached Lice: [0.         0.00136649 0.00464608 0.01284504 0.05165346 1.        ]


,LocalityNumber,Quarter,Disease,Adult Female Lice Category,Movable Lice Category,Attached Lice Category
1,12067,2019-Q3,PD,Very Low,Very Low,Very Low
2,12067,2019-Q3,PD,Very Low,Very Low,Very Low
3,12067,2019-Q3,PD,Very Low,Very Low,Very Low
4,12067,2019-Q3,PD,Very Low,Very Low,Very Low
5,12067,2019-Q3,PD,Very Low,Very Low,Low


In [9]:
# displaying sizes of each bin, to have better view over data
for column in columns_to_bin:
    binned_column = column + ' Category'
    if binned_column in merged_df.columns:
        print(f"Value counts for {binned_column}:")
        print(merged_df[binned_column].value_counts())
        print("\n")


Value counts for Adult Female Lice Category:
Adult Female Lice Category
Very Low     355203
Low          322083
Medium       298847
High         254895
Very High     64298
Name: count, dtype: int64


Value counts for Movable Lice Category:
Movable Lice Category
Very Low     325346
Low          324403
Medium       322369
High         258702
Very High     64506
Name: count, dtype: int64


Value counts for Attached Lice Category:
Attached Lice Category
Very Low     353152
Low          314039
Medium       309741
High         253641
Very High     64753
Name: count, dtype: int64




In [15]:
from mlxtend.frequent_patterns import apriori, association_rules

# Convert the dataset into a one-hot encoded format
merged_onehot = pd.get_dummies(merged_df[['Disease', 'Adult Female Lice Category', 'Movable Lice Category', 'Attached Lice Category']])

# Applying the Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(merged_onehot, min_support=0.06, use_colnames=True)

# Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)

# Display the generated rules
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head()

,antecedents,consequents,support,confidence,lift
0,(Adult Female Lice Category_Very Low),(Movable Lice Category_Very Low),0.157045,0.572701,2.280139
1,(Movable Lice Category_Very Low),(Adult Female Lice Category_Very Low),0.157045,0.625257,2.280139
2,(Attached Lice Category_Very Low),(Adult Female Lice Category_Very Low),0.111332,0.408354,1.489153
3,(Adult Female Lice Category_Very Low),(Attached Lice Category_Very Low),0.111332,0.405996,1.489153
4,(Adult Female Lice Category_Low),(Movable Lice Category_Low),0.088007,0.353940,1.413265


In [16]:
# Calculate associations between Disease and Lice categories
associations = merged_df.groupby('Disease').agg({
    'Adult Female Lice Category': lambda x: x.mode()[0] if not x.mode().empty else None,
    'Movable Lice Category': lambda x: x.mode()[0] if not x.mode().empty else None,
    'Attached Lice Category': lambda x: x.mode()[0] if not x.mode().empty else None
}).reset_index()

associations

,Disease,Adult Female Lice Category,Movable Lice Category,Attached Lice Category
0,ILA,Medium,Very Low,Very Low
1,PD,Very Low,Very Low,Very Low
